In [4]:
import psycopg2
import pandas as pd
import datetime
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler

from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingGridSearchCV

In [5]:
params = params = {
    "host": "localhost",
    "user": "postgres",
    "port": 5432,
    #"password": "şifre" 
}
connection = psycopg2.connect(**params, dbname= "postgres")

In [6]:
df = pd.read_sql("select * from public.spotify_log sl join public.spotify_track st on sl.track_id_clean = st.track_id;", connection)

/Users/hasanenesguray/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [7]:
df.head()

,session_id,session_position,session_length,track_id_clean,skip_1,skip_2,skip_3,not_skipped,context_switch,no_pause_before_play,...,time_signature,valence,acoustic_vector_0,acoustic_vector_1,acoustic_vector_2,acoustic_vector_3,acoustic_vector_4,acoustic_vector_5,acoustic_vector_6,acoustic_vector_7
0,0_00006f66-33e5-4de7-a324-2d18e439fc1e,3,20,t_fc5df5ba-5396-49a7-8b29-35d0d28249e0,False,False,False,True,0,1,...,4,0.373862,-0.742541,0.375599,0.252660,-0.049007,-0.299745,0.063341,-0.486689,0.181604
1,0_00006f66-33e5-4de7-a324-2d18e439fc1e,10,20,t_2af4dfa0-7df3-4b7e-b7ab-353ba48237f9,True,True,True,False,0,1,...,4,0.257672,-0.647221,0.316101,0.251329,-0.041532,-0.252359,0.059971,-0.313696,0.126421
2,0_00006f66-33e5-4de7-a324-2d18e439fc1e,11,20,t_79570b2a-639a-4ec0-9853-71c5299ac44d,True,True,True,False,0,1,...,4,0.037998,-0.573548,0.284808,0.249904,-0.080877,-0.337629,0.116421,-0.337717,0.247187
3,0_00006f66-33e5-4de7-a324-2d18e439fc1e,17,20,t_bff5b6c6-6968-41be-b723-7e860b372975,False,True,True,False,0,1,...,4,0.283693,-0.837439,0.368295,0.271468,0.024429,-0.292806,0.032801,-0.485078,0.183784
4,0_0000a72b-09ac-412f-b452-9b9e79bded8f,2,20,t_0632570b-b4b4-4c3c-b47c-443ec9fbaa11,True,True,True,False,0,1,...,4,0.555482,0.203349,0.247835,-0.136071,-0.440866,0.680082,0.181352,-0.279335,0.001125


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167880 entries, 0 to 167879
Data columns (total 51 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   session_id                       167880 non-null  object 
 1   session_position                 167880 non-null  int64  
 2   session_length                   167880 non-null  int64  
 3   track_id_clean                   167880 non-null  object 
 4   skip_1                           167880 non-null  bool   
 5   skip_2                           167880 non-null  bool   
 6   skip_3                           167880 non-null  bool   
 7   not_skipped                      167880 non-null  bool   
 8   context_switch                   167880 non-null  int64  
 9   no_pause_before_play             167880 non-null  int64  
 10  short_pause_before_play          167880 non-null  int64  
 11  long_pause_before_play           167880 non-null  int64  
 12  hi

In [9]:
df.date.unique()

array(['2018-07-15', '2018-07-14', '2018-07-13', '2018-07-12',
       '2018-06-05', '2018-07-11', '2018-07-16', '2018-07-10',
       '2018-07-04', '2018-07-05', '2018-07-07', '2018-07-06',
       '2018-05-12', '2018-07-09', '2018-06-29', '2018-06-30',
       '2018-06-19', '2018-05-18'], dtype=object)

In [10]:
def weekday_converter(date):
    format = '%Y-%m-%d'
    datetime_date = datetime.datetime.strptime(date,format)
    weekdays_list = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
    return weekdays_list[datetime_date.weekday()]

df = df.assign(weekday=df.date.apply(weekday_converter))
df.head()

,session_id,session_position,session_length,track_id_clean,skip_1,skip_2,skip_3,not_skipped,context_switch,no_pause_before_play,...,valence,acoustic_vector_0,acoustic_vector_1,acoustic_vector_2,acoustic_vector_3,acoustic_vector_4,acoustic_vector_5,acoustic_vector_6,acoustic_vector_7,weekday
0,0_00006f66-33e5-4de7-a324-2d18e439fc1e,3,20,t_fc5df5ba-5396-49a7-8b29-35d0d28249e0,False,False,False,True,0,1,...,0.373862,-0.742541,0.375599,0.252660,-0.049007,-0.299745,0.063341,-0.486689,0.181604,Sunday
1,0_00006f66-33e5-4de7-a324-2d18e439fc1e,10,20,t_2af4dfa0-7df3-4b7e-b7ab-353ba48237f9,True,True,True,False,0,1,...,0.257672,-0.647221,0.316101,0.251329,-0.041532,-0.252359,0.059971,-0.313696,0.126421,Sunday
2,0_00006f66-33e5-4de7-a324-2d18e439fc1e,11,20,t_79570b2a-639a-4ec0-9853-71c5299ac44d,True,True,True,False,0,1,...,0.037998,-0.573548,0.284808,0.249904,-0.080877,-0.337629,0.116421,-0.337717,0.247187,Sunday
3,0_00006f66-33e5-4de7-a324-2d18e439fc1e,17,20,t_bff5b6c6-6968-41be-b723-7e860b372975,False,True,True,False,0,1,...,0.283693,-0.837439,0.368295,0.271468,0.024429,-0.292806,0.032801,-0.485078,0.183784,Sunday
4,0_0000a72b-09ac-412f-b452-9b9e79bded8f,2,20,t_0632570b-b4b4-4c3c-b47c-443ec9fbaa11,True,True,True,False,0,1,...,0.555482,0.203349,0.247835,-0.136071,-0.440866,0.680082,0.181352,-0.279335,0.001125,Sunday


In [11]:
def track_age_calculator(year):
    return 2022-year

df = df.assign(track_age=df.release_year.apply(track_age_calculator))
df[['release_year','track_age']]

,release_year,track_age
0,2018,4
1,2018,4
2,2018,4
3,2018,4
4,1974,48
...,...,...
167875,2015,7
167876,2018,4
167877,2018,4
167878,2018,4


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167880 entries, 0 to 167879
Data columns (total 53 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   session_id                       167880 non-null  object 
 1   session_position                 167880 non-null  int64  
 2   session_length                   167880 non-null  int64  
 3   track_id_clean                   167880 non-null  object 
 4   skip_1                           167880 non-null  bool   
 5   skip_2                           167880 non-null  bool   
 6   skip_3                           167880 non-null  bool   
 7   not_skipped                      167880 non-null  bool   
 8   context_switch                   167880 non-null  int64  
 9   no_pause_before_play             167880 non-null  int64  
 10  short_pause_before_play          167880 non-null  int64  
 11  long_pause_before_play           167880 non-null  int64  
 12  hi

In [13]:
df.drop('session_id', inplace=True, axis=1)
df.drop('track_id_clean', inplace=True, axis=1)
df.drop('skip_1', inplace=True, axis=1)
df.drop('skip_3', inplace=True, axis=1)
df.drop('not_skipped', inplace=True, axis=1)
df.drop('date', inplace=True, axis=1)
df.drop('track_id', inplace=True, axis=1)





In [14]:
df.drop('context_type', inplace=True, axis=1)
df.drop('hist_user_behavior_reason_start', inplace=True, axis=1)
df.drop('hist_user_behavior_reason_end', inplace=True, axis=1)
df.drop('weekday', inplace=True, axis=1)
df.drop('mode', inplace=True, axis=1)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167880 entries, 0 to 167879
Data columns (total 41 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   session_position               167880 non-null  int64  
 1   session_length                 167880 non-null  int64  
 2   skip_2                         167880 non-null  bool   
 3   context_switch                 167880 non-null  int64  
 4   no_pause_before_play           167880 non-null  int64  
 5   short_pause_before_play        167880 non-null  int64  
 6   long_pause_before_play         167880 non-null  int64  
 7   hist_user_behavior_n_seekfwd   167880 non-null  int64  
 8   hist_user_behavior_n_seekback  167880 non-null  int64  
 9   hist_user_behavior_is_shuffle  167880 non-null  bool   
 10  hour_of_day                    167880 non-null  int64  
 11  premium                        167880 non-null  bool   
 12  duration                      

In [16]:
df["skip_2"] = df["skip_2"].astype(int)
df["hist_user_behavior_is_shuffle"] = df["hist_user_behavior_is_shuffle"].astype(int)
df["premium"] = df["premium"].astype(int)

In [17]:
target_count = df.skip_2.value_counts(normalize=True)
target_count

1    0.517179
0    0.482821
Name: skip_2, dtype: float64

In [18]:
columns = df.columns.tolist()
columns = [columns[2]] + columns[0:2] + columns[3:]
df = df[columns]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167880 entries, 0 to 167879
Data columns (total 41 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   skip_2                         167880 non-null  int64  
 1   session_position               167880 non-null  int64  
 2   session_length                 167880 non-null  int64  
 3   context_switch                 167880 non-null  int64  
 4   no_pause_before_play           167880 non-null  int64  
 5   short_pause_before_play        167880 non-null  int64  
 6   long_pause_before_play         167880 non-null  int64  
 7   hist_user_behavior_n_seekfwd   167880 non-null  int64  
 8   hist_user_behavior_n_seekback  167880 non-null  int64  
 9   hist_user_behavior_is_shuffle  167880 non-null  int64  
 10  hour_of_day                    167880 non-null  int64  
 11  premium                        167880 non-null  int64  
 12  duration                      

In [19]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], 
                                                    test_size = 0.3, random_state=42)

In [20]:
print(X_train.shape) # Eğitim için %70 
print(X_test.shape) # Test için %30

(117516, 40)
(50364, 40)


In [21]:
print(y_train.shape)
print(y_test.shape)

(117516,)
(50364,)


In [ ]:
scale = StandardScaler()

X_train_transformed = scale.fit_transform(X_train)
X_test_transformed = scale.transform(X_test)


knn = KNeighborsClassifier(n_neighbors=7,weights='uniform',metric='minkowski',n_jobs=-1)
knn.fit(X_train_transformed, y_train)
y_pred = knn.predict(X_test_transformed)

print(metrics.accuracy_score(y_test, y_pred))

In [ ]:
#knn = KNeighborsClassifier(n_neighbors=1)
#knn.fit(X_train, y_train)
#y_pred = knn.predict(X_test)

#print(metrics.accuracy_score(y_test, y_pred))

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7,weights='uniform',metric='minkowski',n_jobs=-1)
scores = cross_val_score(knn, df.iloc[:, 1:], df.iloc[:, 0], cv=10, scoring='accuracy') # Veri setini 10 farklı formatta hazırla
print(scores)

In [ ]:
# Elde ettiğimiz bu accuracy değerlerinden bir çıkarım yapabilmek için aritmetik ortalama almak oldukça mantıklı bir yöntem
print(scores.mean())

In [ ]:
scale = StandardScaler()

X_train_transformed = scale.fit_transform(X_train)
X_test_transformed = scale.transform(X_test)


knn = KNeighborsClassifier(n_neighbors=15,weights='uniform',metric='minkowski',n_jobs=-1)
knn.fit(X_train_transformed, y_train)
y_pred = knn.predict(X_test_transformed)

print(metrics.accuracy_score(y_test, y_pred))

In [ ]:
scale = StandardScaler()

X_train_transformed = scale.fit_transform(X_train)
X_test_transformed = scale.transform(X_test)


knn = KNeighborsClassifier(n_neighbors=21,weights='uniform',metric='minkowski',n_jobs=-1)
knn.fit(X_train_transformed, y_train)
y_pred = knn.predict(X_test_transformed)

print(metrics.accuracy_score(y_test, y_pred))

In [ ]:
scale = StandardScaler()

X_train_transformed = scale.fit_transform(X_train)
X_test_transformed = scale.transform(X_test)


knn = KNeighborsClassifier(n_neighbors=25,weights='uniform',metric='minkowski',n_jobs=-1)
knn.fit(X_train_transformed, y_train)
y_pred = knn.predict(X_test_transformed)

print(metrics.accuracy_score(y_test, y_pred))

In [ ]:
scale = StandardScaler()

X_train_transformed = scale.fit_transform(X_train)
X_test_transformed = scale.transform(X_test)


knn = KNeighborsClassifier(n_neighbors=31,weights='uniform',metric='minkowski',n_jobs=-1)
knn.fit(X_train_transformed, y_train)
y_pred = knn.predict(X_test_transformed)

print(metrics.accuracy_score(y_test, y_pred))

In [ ]:
scale = StandardScaler()

X_train_transformed = scale.fit_transform(X_train)
X_test_transformed = scale.transform(X_test)


knn = KNeighborsClassifier(n_neighbors=33,weights='uniform',metric='minkowski',n_jobs=-1)
knn.fit(X_train_transformed, y_train)
y_pred = knn.predict(X_test_transformed)

print(metrics.accuracy_score(y_test, y_pred))

In [ ]:
# Hangi parametrelerin değerlendirileceğini tanımlama
k_range = list(range(3,9,2))
weight_options = ['uniform', 'distance']
metric_options = ['euclidean', 'manhattan', 'minkowski']

In [ ]:
param_dist = dict(n_neighbors=k_range, weights=weight_options, metric=metric_options)

In [ ]:
rand = HalvingGridSearchCV(knn, param_dist, scoring='accuracy', n_jobs=-1, min_resources="exhaust",factor = 3)
rand.fit(df.iloc[:, 1:], df.iloc[:, 0])
rand.cv_results_

In [ ]:
def examine_results(model_grid):
    print("rank_test_score", model_grid.cv_results_['rank_test_score'])
    print("mean_test_score", model_grid.cv_results_['mean_test_score'])
    
examine_results(rand)

In [ ]:
print('Best Score:', rand.best_score_, 'Best Parameters:', rand.best_params_)